<a href="https://colab.research.google.com/github/fazal735/DL_A2/blob/main/DL_A2(partB).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torchvision import models
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader,SubsetRandomSampler

from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')
train_data_dir = '/content/drive/MyDrive/inaturalist_12K/train'
test_data_dir = '/content/drive/MyDrive/inaturalist_12K/val'

Mounted at /content/drive


In [13]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [3]:
#data loading
data_dir='/content/drive/MyDrive/inaturalist_12K/train'

def data_load(data_dir,data_augumentation):
    # Define data transformations
    if data_augumentation == 'Yes':
        transform = transforms.Compose([
            transforms.RandomResizedCrop(224),  # Randomly crop and resize to 224x224
            transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Randomly adjust brightness, contrast, saturation, and hue
            transforms.RandomRotation(20),  # Randomly rotate the image by up to 20 degrees
            transforms.ToTensor(),  # Convert images to PyTorch tensors
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize image tensors
        ])

    else:
        transform = transforms.Compose([
            transforms.Resize((224, 224)),  # Resize images to 224x224 (compatible with the CNN input size)
            transforms.ToTensor(),  # Convert images to PyTorch tensors
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize image tensors
        ])

    # Load the dataset using ImageFolder and apply transformations
    training_data = ImageFolder(root=data_dir, transform=transform)
    # Splitting train dataset into training and validation indices
    train_index, val_index = train_test_split(list(range(len(training_data))), test_size=0.2, random_state=42)
    # Create DataLoader instances for training and validation sets


    random_train_sample = SubsetRandomSampler(train_index)
    train_data = DataLoader(training_data, batch_size=32, sampler=random_train_sample)


    random_val_sample = SubsetRandomSampler(val_index)
    validation_data = DataLoader(training_data, batch_size=32, sampler=random_val_sample)


    return train_data, validation_data

In [15]:

def training_on_trained_model(model,data):
  criterion=torch.nn.CrossEntropyLoss()
  optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

  model.train()

  training_loss=0.00
  correct_label=0
  total_train=0

  for inputs,labels in data:
    inputs,labels=inputs.to(device),labels.to(device)
    optimizer.zero_grad()
    outputs=model(inputs)
    loss=criterion(outputs,labels)
    loss.backward()
    optimizer.step()

    training_loss+=loss.item()
    _,predicted=torch.max(outputs.data,1)
    total_train+=labels.size(0)
    correct_label+=(predicted==labels).sum().item()

    #accuracies
    avg_loss=training_loss/len(data)
    training_accuracy=100*correct_label/total_train
    return avg_loss,training_accuracy

def validation_on_trained_model(model,valdata):
  model.eval()
  correct_label=0
  total_val=0
  with torch.no_grad():
    for inputs,labels in valdata:
      inputs,labels=inputs.to(device),labels.to(device)
      outputs=model(inputs)
      _,predicted=torch.max(outputs.data,1)
      total_val+=labels.size(0)
      correct_label+=(predicted==labels).sum().item()

  validation_accuracy=100*correct_label/total_val
  return validation_accuracy


In [5]:
def trained_model(model_name,freeze_percentage,freeze_all_but_last_layer):

    if model_name == 'resnet':
        pretrained_model = models.resnet50(pretrained=True)
    elif model_name == 'inception':
        pretrained_model = models.inception_v3(pretrained=True)

    nodes_fc = pretrained_model.fc.in_features
    pretrained_model.fc = nn.Linear(nodes_fc, 10)  # Change 10 to the number of classes

    total_layers = sum(1 for _ in pretrained_model.children())
    num_freeze = int(total_layers * freeze_percent)

    if freez_all_but_last_layer == 'Yes':
        for name, parameter in pretrained_model.named_parameters():
            if not name.startswith('fc'):  # Exclude the last layer named 'fc'
                parameter.requires_grad = False

    if freez_all_but_last_layer == 'No':
        count = 0
        for name, name_chld in pretrained_model.named_children():
            if count < num_freeze:
                for parameter in name_chld.parameters():
                    parameter.requires_grad = False  # Freeze the layer
            else:
                for parameter in name_chld.parameters():
                    parameter.requires_grad = True  # Make the layer trainable
            count += 1
    return pretrained_model

# Example usage
freeze_percent = 0.25  # Freeze 25% of layers
freez_all_but_last_layer = 'No'
model_name = 'resnet'
model = trained_model(model_name,freeze_percent,freez_all_but_last_layer)
print(model)



/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 153MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
# Function to training model
def model_train(model, train_data, val_data,epochs):

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        avg_loss, train_accuracy = training_on_trained_model(model, train_data)
        # Print training loss and accuracy
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {avg_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')
        wandb.log({'Train loss': avg_loss})
        wandb.log({'Train accuracy': train_accuracy})


        # Validation loop
        val_accuracy = validation_on_trained_model(model, val_data)
        # Print validation accuracy
        print(f'Epoch {epoch+1}/{epochs}, Validation Accuracy: {val_accuracy:.2f}%')
        wandb.log({'val_accuracy': val_accuracy})
        wandb.log({'epoch': epoch})

    print('Training complete!')

In [7]:
!pip install wandb

In [8]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mfazal735 (mfazal735-iit-madras-foundation) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [9]:
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'epoch': {
            'values': [6]
        },
        'freezing_percentage': {
            'values': [0.8,0.9]
        },
        'model': {
            'values': [ 'resnet','inception']
        }
    }
}


sweep_id = wandb.sweep(sweep_config, project='DL_A2(partB)')

Create sweep with ID: liu31voa
Sweep URL: https://wandb.ai/mfazal735-iit-madras-foundation/DL_A2%28partB%29/sweeps/liu31voa


In [16]:

def main():

    with wandb.init() as run:
        run_name="ep"+str(wandb.config.epoch)+"_fp-"+str(wandb.config.freezing_percentage)+"_model-"+(wandb.config.model)
        wandb.run.name=run_name
# wandb.config.freezing_percentage   "_fp-"+str(wandb.config.freezing_percentage)+

        model = trained_model(model_name = wandb.config.model,freeze_percentage=wandb.config.freezing_percentage ,freeze_all_but_last_layer='No')
        model = model.to(device)

        data_dir = '/content/drive/MyDrive/inaturalist_12K/train'
        train, validation = data_load(data_dir,data_augumentation= 'No')

        model_train(model,train,validation,epochs=wandb.config.epoch)

wandb.agent(sweep_id, function= main,count= 1)

wandb: Agent Starting Run: 746fjt5e with config:
wandb: 	epoch: 6
wandb: 	freezing_percentage: 0.8
wandb: 	model: resnet


Epoch 1/6, Train Loss: 0.0097, Train Accuracy: 9.38%
Epoch 1/6, Validation Accuracy: 17.05%
Epoch 2/6, Train Loss: 0.0092, Train Accuracy: 34.38%
Epoch 2/6, Validation Accuracy: 21.00%
Epoch 3/6, Train Loss: 0.0093, Train Accuracy: 31.25%
Epoch 3/6, Validation Accuracy: 15.60%
Epoch 4/6, Train Loss: 0.0077, Train Accuracy: 43.75%
Epoch 4/6, Validation Accuracy: 17.75%
Epoch 5/6, Train Loss: 0.0115, Train Accuracy: 31.25%
Epoch 5/6, Validation Accuracy: 14.55%
Epoch 6/6, Train Loss: 0.0093, Train Accuracy: 31.25%
Epoch 6/6, Validation Accuracy: 18.15%
Training complete!


Train accuracy,▁▆▅█▅▅
Train loss,▅▄▄▁█▄
epoch,▁▂▄▅▇█
val_accuracy,▄█▂▄▁▅
Train accuracy,31.25
Train loss,0.00932
epoch,5
val_accuracy,18.15
